In [1]:
import gspread
import random
import math
import smtplib
import pandas as pd
from google.colab import auth
from oauth2client.service_account import ServiceAccountCredentials
from gspread.exceptions import APIError
from gspread.models import Spreadsheet
from google.auth import default
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
!pip install py3-validate-email
!pip install py3dns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 KB 2.4 MB/s eta 0:00:00
  Created wheel for py3-validate-email: filename=py3_validate_email-1.0.5.post1-py3-none-any.whl size=34199 sha256=1140ce5980f7cf5d7d780844d84256f068a370eb10cc78d86e1dfbff99848b04
  Stored in directory: /root/.cache/pip/wheels/50/23/e6/2fc84da654e7eb40bd80d681d93e1bc87185fbf695cb79ef1f
Successfully built py3-validate-email
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.25.1 requires idna<3,>=2.5, but you have idna 3.4 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us

In [3]:
# Autenticação com a API do Google
auth.authenticate_user()
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds, _ = default()
client = gspread.authorize(creds)

In [4]:
# Obtenção da planilha com as respostas
sheet = client.open('Check-in Rede Plenus (respostas)').sheet1

# Obtenção dos nomes dos participantes
participantes = [{'Nome': r['Nome Completo'], 'Email': r['E-mail'], 'Telefone': r['Telefone'], 'Grupo': r['Grupo']} for r in sheet.get_all_records()]

In [5]:
import random

# Dicionário para armazenar os grupos
grupos = {
    "Azul": [],
    "Verde": [],
    "Vermelho": []
}

# Cálculo da quantidade de participantes por grupo
quantidade_por_grupo = len(participantes) // len(grupos)

# Verificação para garantir que todos os grupos tenham membros
if len(participantes) % len(grupos) != 0:
    quantidade_por_grupo += 1

# Sorteio dos participantes para cada grupo
for participante in participantes:
    grupo = random.choice(list(grupos.keys()))

    while len(grupos[grupo]) >= quantidade_por_grupo:
        grupo = random.choice(list(grupos.keys()))

    grupos[grupo].append(participante)

In [6]:
grupos

{'Azul': [{'Nome': 'Pedro Araujo',
   'Email': 'graziellamilene@gmail.com',
   'Telefone': 128032789372,
   'Grupo': ''},
  {'Nome': 'Panda',
   'Email': 'modav97773@wiroute.com',
   'Telefone': 1111112122,
   'Grupo': ''},
  {'Nome': 'Anne Zimmerman',
   'Email': 'mitchelljoshua@example.net',
   'Telefone': '683-504-6659x550',
   'Grupo': ''},
  {'Nome': 'George Smith',
   'Email': 'john97@example.org',
   'Telefone': '5.454.717.924',
   'Grupo': ''},
  {'Nome': 'James Adams',
   'Email': 'sdalton@example.com',
   'Telefone': '205-920-0932',
   'Grupo': ''},
  {'Nome': 'Dan Ramos',
   'Email': 'christopher65@example.net',
   'Telefone': '+1-849-263-6622x43532',
   'Grupo': ''},
  {'Nome': 'Dana Espinoza',
   'Email': 'keithmitchell@example.org',
   'Telefone': '001-526-804-0993x3372',
   'Grupo': ''},
  {'Nome': 'Pedro Grimes',
   'Email': 'paigebolton@example.net',
   'Telefone': '001-147-200-7451x025',
   'Grupo': ''},
  {'Nome': 'Catherine Bradley',
   'Email': 'xjohnson@example.co

In [7]:
resultado_sheet = client.create('Resultado do sorteio da Gincana da Rede Plenus')

# Adicionar o resultado do sorteio à nova planilha
for i, (grupo, participantes_grupo) in enumerate(grupos.items()):
    grupo_sheet = resultado_sheet.add_worksheet(title=grupo, rows=len(participantes_grupo)+1, cols=len(participantes_grupo[0])-1)
    headers = list(participantes_grupo[0].keys())
    headers.remove('Grupo')
    print(f'Adicionando os grupos na planilha: ', grupo_sheet)
    grupo_sheet.insert_row(headers, 1)
    for j, participante in enumerate(participantes_grupo):
        values = list(participante.values())
        del values[-1]  # remove o valor do grupo da lista de valores
        grupo_sheet.insert_row(values, j+2)

padrao_sheet = resultado_sheet.worksheet('Página1')
resultado_sheet.del_worksheet(padrao_sheet)

Adicionando os grupos na planilha:  <Worksheet 'Azul' id:1338778175>
Adicionando os grupos na planilha:  <Worksheet 'Verde' id:1292176061>
Adicionando os grupos na planilha:  <Worksheet 'Vermelho' id:1342352135>


{'spreadsheetId': '1jsxAPFTpKGw41XT03npW-VzuuPx0DfVopPcam5WuGcU',
 'replies': [{}]}

In [12]:
import DNS
from validate_email import validate_email

link_azul = "https://chat.whatsapp.com/xxxxxxxxxxxxxxx"
link_verde = "https://chat.whatsapp.com/yyyyyyyyyyyyyyy"
link_vermelho = "https://chat.whatsapp.com/zzzzzzzzzzzzzzz"

# Envio do e-mail para cada participante informando a qual equipe pertence
for grupo, participantes in grupos.items():
    for participante in participantes:
        nome = participante["Nome"]
        email = participante["Email"]
        
        if grupo == 'Azul':
            link = link_azul
        elif grupo == 'Verde':
            link = link_verde
        else:
            link = link_vermelho

        mensagem = f"Olá {nome},\n\nVocê foi sorteado para o Grupo {grupo}.\n\nO link para o grupo da sua equipe é:\n{link}\n\nAtenciosamente,\nRede Plenus"
        
        # Configuração da mensagem
        msg = MIMEMultipart()
        msg['From'] = 'ralfferreira995@gmail.com'
        msg['To'] = email
        msg['Subject'] = "Resultado do sorteio da Gincana da Rede Plenus"
        msg.attach(MIMEText(mensagem, 'plain'))

        is_valid = validate_email(
            email
        )
        
        if is_valid:
            # Conexão com o servidor de e-mail e envio da mensagem
            servidor_smtp = smtplib.SMTP('smtp.gmail.com', 587)
            servidor_smtp.starttls()
            servidor_smtp.login('ralfferreira995@gmail.com', 'dgbqzvwnobznhxsu')
            print(f"Enviando e-mail para: {email}")
            servidor_smtp.sendmail('ralfferreira995@gmail.com', email, msg.as_string().encode('utf-8'))
            servidor_smtp.quit()
        else:
          print(f'E-mail inválido: {email}')

Enviando e-mail para: graziellamilene@gmail.com
Enviando e-mail para: modav97773@wiroute.com
E-mail inválido: mitchelljoshua@example.net
E-mail inválido: john97@example.org
E-mail inválido: sdalton@example.com
E-mail inválido: christopher65@example.net
E-mail inválido: keithmitchell@example.org
E-mail inválido: paigebolton@example.net
E-mail inválido: xjohnson@example.com
E-mail inválido: heathercarter@example.com
E-mail inválido: chrisperry@example.org
E-mail inválido: randy98@example.org
E-mail inválido: medinamatthew@example.org
E-mail inválido: joelnelson@example.com
E-mail inválido: gonzalezmatthew@example.com
E-mail inválido: fbuckley@example.com
E-mail inválido: whudson@example.com
E-mail inválido: fsmith@example.net
E-mail inválido: savannah97@example.org
E-mail inválido: carrie79@example.org
E-mail inválido: scottdean@example.org
E-mail inválido: dstewart@example.org
E-mail inválido: kevin46@example.com
E-mail inválido: smithgerald@example.net
E-mail inválido: laurenrodriguez@